### imports

In [32]:
#Imports which i took from the model_evaluation notebook 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dill as pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import math  
from sklearn import metrics

from sklearn.model_selection import learning_curve

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.inspection import permutation_importance
import warnings
warnings.simplefilter(action="ignore")

# model iterations (dummy,LinearRegression)

## baseline first iteration - score:0.046

In [2]:
#importing the file
iter1 = pd.read_pickle("data/iter1.pkl")  

In [3]:
iter1

,bloc_id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,2072,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,0
1,10600,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,1
2,2494,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,0
3,4284,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,1
4,16541,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,0
...,...,...,...,...,...,...,...,...,...,...,...
16507,1099,-121.90,39.59,20.0,1465.0,278.0,745.0,250.0,3.0625,93800.0,0
16508,18898,-122.25,38.11,49.0,2365.0,504.0,1131.0,458.0,2.6133,103100.0,3
16509,11798,-121.22,38.92,19.0,2531.0,461.0,1206.0,429.0,4.4958,192600.0,0
16510,6637,-118.14,34.16,39.0,2776.0,840.0,2546.0,773.0,2.5750,153500.0,1


In [4]:
#defining x and y for the dummy model
y = iter1["median_house_value"]
x = iter1.drop("median_house_value", axis=1)

In [5]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(x, y)

# calculate the score
dummy_clf.score(x, y)

0.04620881782945736

## second iteration model - score: 0.6350 (the lineareg function is here)

In [6]:
iter2 = pd.read_pickle("data/iter2.pkl")
iter2

,bloc_id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,2072,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,0
1,10600,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,1
2,2494,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,0
3,4284,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,1
4,16541,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,0
...,...,...,...,...,...,...,...,...,...,...,...
16507,1099,-121.90,39.59,20.0,1465.0,278.0,745.0,250.0,3.0625,93800.0,0
16508,18898,-122.25,38.11,49.0,2365.0,504.0,1131.0,458.0,2.6133,103100.0,3
16509,11798,-121.22,38.92,19.0,2531.0,461.0,1206.0,429.0,4.4958,192600.0,0
16510,6637,-118.14,34.16,39.0,2776.0,840.0,2546.0,773.0,2.5750,153500.0,1


In [30]:
#making a function to do everything i need in a Linear regression
def Lineareg(data,target):
    #linear regression
    y = data[target]
    X = data.drop(target, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)
    model = LinearRegression()
    model.fit(X_train, y_train)
    print(f'initial model score is {model.score(X_test, y_test)}')
    #cross validation
    K = []
    total_time = []
    score = []
    for k in range(2,20):
        cross_val_results = cross_validate(model, X, y, cv=k)
        total_time.append(sum(cross_val_results['fit_time'])+sum(cross_val_results['score_time']))
        K.append(k)
        score.append(cross_val_results['test_score'].mean())
    #wrote these 2 lines to select the best cross validate value
    best_cv = pd.DataFrame({'K': K,'score': score})
    cross = best_cv.query(f'score=={max(score)}')['K'].item()
    cv_results = cross_validate(model,X, y,cv=cross, scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_error'))
    r2 = cv_results['test_r2'].mean()
    rmse = cv_results['test_neg_root_mean_squared_error'].mean()
    print(f'r2: {r2}')
    print(f'rmse: {rmse}')

In [34]:
#export as pickle
with open('model.pkl', 'wb') as file:
        pickle.dump(Lineareg,file)

In [31]:
Lineareg(iter2,"median_house_value")

initial model score is 0.6350614875943679
r2: 0.63881425552166
rmse: -69205.21355025069


## third iteration model - score:  0.6185

In [9]:
iter3 = pd.read_pickle("data/iter3.pkl")
Lineareg(iter3,"median_house_value")

initial model score is 0.6185272172234932
r2: 0.6081291779140877
rmse: -58708.93719557278


## fourth iteration model - score: 0.6360

In [10]:
iter4 = pd.read_pickle("data/iter4.pkl")
Lineareg(iter4,"median_house_value")

initial model score is 0.6360002572868537
r2: 0.6397597795082213
rmse: -69154.2193492309


## fifth iteration model - score: 0.6169

In [11]:
iter5 = pd.read_pickle("data/iter5.pkl")
Lineareg(iter5,"median_house_value")

initial model score is 0.6169227555786434
r2: 0.6177569431410561
rmse: -56656.626845464445


## sixth iteration model - score: 0.6360

In [12]:
iter6 = pd.read_pickle("data/iter6.pkl")
Lineareg(iter6,"median_house_value")

initial model score is 0.6360861449922407
r2: 0.6398649288734601
rmse: -69144.16949423551


## seventh iteration model - score: 0.6169

In [13]:
iter7 = pd.read_pickle("data/iter7.pkl")
Lineareg(iter7,"median_house_value")

initial model score is 0.6169267334476197
r2: 0.6179398130389817
rmse: -56642.991604169285


## eigth iteration model - score: 0.6450

In [14]:
iter8 = pd.read_pickle("data/iter8.pkl")
Lineareg(iter8,"median_house_value")

initial model score is 0.6450309914821523
r2: 0.6467681927019666
rmse: -68460.69034423704


## ninth iteration model - score: none

In [15]:
#can't work because of the string type of the binned house median age
# iter9 = pd.read_pickle("data/iter9.pkl")
# Lineareg(iter9,"median_house_value")

## tenth iteration model - score: 0.6498 best*

In [16]:
iter10 = pd.read_pickle("data/iter10.pkl")
Lineareg(iter10,"median_house_value")

initial model score is 0.6498355886823779
r2: 0.6496012523146573
rmse: -68161.92722222565


## eleventh iteration model - score: 0.6498

In [17]:
iter11 = pd.read_pickle("data/iter11.pkl")
Lineareg(iter11,"median_house_value")

initial model score is 0.6498355886823752
r2: 0.6496012523146564
rmse: -68161.92722222574


## twelth iteration model - score: 0.6498

In [18]:
iter12 = pd.read_pickle("data/iter12.pkl")
Lineareg(iter12,"median_house_value")

initial model score is 0.6498355886823753
r2: 0.6496012523146564
rmse: -68161.92722222574


## thirtheenth iteration model - score: 0.5331

In [19]:
#it's so much worse after normalizing, why ???
iter13 = pd.read_pickle("data/iter13.pkl")
Lineareg(iter13,"median_house_value")

initial model score is 0.5331797419092961
r2: 0.5338429008155616
rmse: -78588.2948120826


## fourteenth iteration model - score: 

In [20]:
# iter14 = pd.read_pickle("data/iter14.pkl")
# Lineareg(iter14,"median_house_value")

# KNN - wanted to try other models so i see what i can get

## KNN function

In [21]:
#8 neighboors seems to give the most accuracy for the normalized iteration
def KNN(data,target):
    y = data[target]
    X = data.drop(target, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)
    knn = KNeighborsClassifier(n_neighbors=8)

    #Train the model using the training sets
    knn.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = knn.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

## iterations, the results are very bad, i will try to understand what is missing

In [22]:
#best Lineareg iteration
KNN(iter10,"median_house_value")

Accuracy: 0.03088413403310456


In [23]:
#using the normalized data apparently is better
#and looking at the accuracy, it is
KNN(iter13,"median_house_value")

Accuracy: 0.03552684699232943


# DecisionTree - same as KNN

In [24]:
def decisiontree(data,target):
    y = data[target]
    X = data.drop(target, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)
    clf = DecisionTreeClassifier()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [25]:
decisiontree(iter10,"median_house_value")

Accuracy: 0.026846992329430764


In [26]:
decisiontree(iter12,"median_house_value")

Accuracy: 0.027250706499798143


# random forest - doesn't work at all

In [27]:
def rf(data,target):
    y = data[target]
    X = data.drop(target, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)
    clf=RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [28]:
# rf(iter10,"median_house_value")

In [29]:
# rf(iter12,"median_house_value")